In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip -q "/content/gdrive/MyDrive/Colab/Assignment/Data/archive.zip" -d /content/Data

In [ ]:
!pip install -U keras-tuner

In [3]:
from tensorflow.keras.layers.experimental import preprocessing as pl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from kerastuner.tuners import BayesianOptimization
import tensorflow.keras.layers as l
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os

In [4]:
# Hyperparameters and directories

GDRIVE_PROJ_DIR = "/content/gdrive/MyDrive/Colab/Assignment"
TRAIN_DIR = "/content/Data/tiny-imagenet-200/train"
TEST_DIR = "/content/Data/tiny-imagenet-200/val"

epochs         =  10
img_size       =  64
batch_size     =  1024
learning_rate  =  1e-3


In [11]:
def build_model(hp):
    l2_reg_rate    = hp.Float("regularization_rate", 0.0001, 0.005, step=0.0002)
    dropout_scaler = hp.Float("dropout_rate", 1, 3, step=0.25)
    use_moving_avg = hp.Choice("optimizer", [True,False])       

    # Build BackBone
    backbone = EfficientNetB0(include_top=False, weights=None, input_shape=(img_size,img_size,3))
    for layer in backbone.layers:
        layer_name = layer.name
        if "conv" in layer_name:
            layer._activity_regularizer=tf.keras.regularizers.L2(l2=l2_reg_rate)
        if "drop" in layer_name:
            layer.rate *= dropout_scaler
        
    # Build Model
    Augument_pipeline = tf.keras.Sequential([
                                            pl.RandomRotation(factor=0.15),
                                            pl.RandomTranslation(height_factor=0.1, width_factor=0.1),
                                            pl.RandomFlip("horizontal"),
                                            pl.RandomContrast(factor=0.1),
                                            ], name="Image_Augumentation")

    model = tf.keras.Sequential([
                                l.Input(shape = (img_size,img_size,3)),
                                Augument_pipeline,
                                backbone,
                                l.GlobalAveragePooling2D(name="Global_Average_pooling"),
                                l.Dense(200, activation="softmax", name="Classifier")
                                ], name="ImageNet_Tiny_Classifier") 
    
    # Compile Model
    loss = tf.keras.losses.CategoricalCrossentropy()
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    if use_moving_avg:
        opt = tfa.optimizers.MovingAverage(opt)
    metrics = [tf.keras.metrics.CategoricalAccuracy()]

    model.compile(loss=loss, optimizer=opt, metrics=metrics)
    return model

In [14]:
train_csv = pd.read_csv(os.path.join(GDRIVE_PROJ_DIR, 'train.csv'))
train_csv.head()

,image_path,class_code,class_name
0,n03126707/images/n03126707_344.JPEG,n03126707,crane
1,n02437312/images/n02437312_328.JPEG,n02437312,"Arabian camel, dromedary, Camelus dromedarius"
2,n02113799/images/n02113799_124.JPEG,n02113799,standard poodle
3,n03980874/images/n03980874_261.JPEG,n03980874,poncho
4,n03649909/images/n03649909_208.JPEG,n03649909,"lawn mower, mower"


In [15]:
# Creating input Pipeline
datagen = ImageDataGenerator(validation_split=0.2)
train_gen = datagen.flow_from_dataframe(
    train_csv, directory=TRAIN_DIR, x_col='image_path', y_col='class_name',
    weight_col=None, target_size=(img_size, img_size), class_mode='categorical', 
    batch_size=batch_size, shuffle=True,subset="training")
val_gen = datagen.flow_from_dataframe(
    train_csv, directory=TRAIN_DIR, x_col='image_path', y_col='class_name',
    weight_col=None, target_size=(img_size, img_size), class_mode='categorical', 
    batch_size=batch_size, shuffle=False,subset="validation")

Found 80000 validated image filenames belonging to 200 classes.
Found 20000 validated image filenames belonging to 200 classes.


In [16]:
tuner = BayesianOptimization(build_model, objective="val_loss", max_trials=10, seed=27)
tuner.search_space_summary()

Search space summary
Default search space size: 3
regularization_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.005, 'step': 0.0002, 'sampling': None}
dropout_rate (Float)
{'default': 1.0, 'conditions': [], 'min_value': 1.0, 'max_value': 3.0, 'step': 0.25, 'sampling': None}
optimizer (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}


In [17]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0,
                                                mode='auto', baseline=None, restore_best_weights=True)
            ]

In [ ]:
tuner.search(x=train_gen, validation_data=val_gen,
    steps_per_epoch=train_gen.samples//batch_size,
    validation_steps=val_gen.samples//batch_size,
    epochs=epochs, verbose=1, shuffle=True,
    callbacks=callbacks,)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
regularization_...|0.0033            |?                 
dropout_rate      |1.25              |?                 
optimizer         |1                 |?                 

Epoch 1/10
 3/78 [>.............................] - ETA: 35:59 - loss: 1193.6655 - categorical_accuracy: 0.0057